In [117]:
import pandas as pd
import os
from sklearn import preprocessing
from collections import deque
import random
import numpy as np
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization, LSTM
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

In [118]:
seq_len = 60
future_period_predict = 3
ratio_to_predict = "LTC-USD"
epochs = 10
batch_size = 64
name = f"{seq_len}-SEQ-{future_period_predict}-PRED-{int(time.time())}"

def classify(current, future):
    if float(future)> float(current):
        return 1
    else:
        return 0

In [119]:
def preprocess_df(df):
    df =df.drop('future', 1)
    for col in df.columns:
        if col != "target":
            df[col] = df[col].pct_change()
            df.dropna(inplace=True)
            df[col]=preprocessing.scale(df[col].values)

    df.dropna(inplace=True)

    sequential_data = []
    prev_days = deque(maxlen = seq_len) 
    print(df.head())

    for i in df.values:
        prev_days.append([n for n in i[:-1]])
        if len(prev_days) == seq_len:
            sequential_data.append([np.array(prev_days), i[-1]])
    random.shuffle(sequential_data)

    buys = []
    sells = []

    for seq, target in sequential_data:
        if target == 0:
            sells.append([seq, target])
        elif target == 1:
            buys.append([seq, target])

    random.shuffle(buys)
    random.shuffle(sells)

    lower = min(len(buys), len(sells))
    buys = buys[:lower]
    sells = sells[:lower]

    sequential_data = buys + sells
    random.shuffle(sequential_data)

    X = []
    y = []

    for seq, target in sequential_data:
        X.append(seq)
        y.append(target)

    return np.array(X), np.array(y)


In [120]:
main_df = pd.DataFrame()

ratios = ["BTC-USD", "LTC-USD", "ETH-USD", "BCH-USD"]
for ratio in ratios:
    dataset = f"crypto_data/{ratio}.csv"

    df = pd.read_csv(dataset, names = ["time", "low", "high", "open", "close", "volume"])
    #print(df.head())
    df.rename( columns = {"close": f"{ratio}_close", "volume": f"{ratio}_volume"}, inplace=True)

    df.set_index("time", inplace= True)
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]
    #print(df.head())

    if len(main_df) == 0:
        main_df = df
    else:
        main_df = main_df.join(df)
print(main_df.head)


<bound method NDFrame.head of             BTC-USD_close  BTC-USD_volume  LTC-USD_close  LTC-USD_volume  \
time                                                                       
1528968660    6489.549805        0.587100      96.580002        9.647200   
1528968720    6487.379883        7.706374      96.660004      314.387024   
1528968780    6479.410156        3.088252      96.570000       77.129799   
1528968840    6479.410156        1.404100      96.500000        7.216067   
1528968900    6479.979980        0.753000      96.389999      524.539978   
...                   ...             ...            ...             ...   
1535214960    6713.140137        0.769891      58.020000        6.434783   
1535215020    6714.520020        1.002652      58.009998        7.301921   
1535215080    6714.520020        1.021925      58.020000       23.802017   
1535215140    6715.000000        3.645508      58.020000        6.953497   
1535215200    6715.000000        0.513560      58.080002  

In [121]:
main_df["future"] = main_df[f"{ratio_to_predict}_close"].shift(-future_period_predict)
print(main_df[[f"{ratio_to_predict}_close", "future"]].head())

            LTC-USD_close     future
time                                
1528968660      96.580002  96.500000
1528968720      96.660004  96.389999
1528968780      96.570000  96.519997
1528968840      96.500000  96.440002
1528968900      96.389999  96.470001


In [122]:
main_df['target'] = list(map(classify, main_df[f"{ratio_to_predict}_close"], main_df["future"]))
print(main_df[[f"{ratio_to_predict}_close", "future", "target"]].head(10))

            LTC-USD_close     future  target
time                                        
1528968660      96.580002  96.500000       0
1528968720      96.660004  96.389999       0
1528968780      96.570000  96.519997       0
1528968840      96.500000  96.440002       0
1528968900      96.389999  96.470001       1
1528968960      96.519997  96.400002       0
1528969020      96.440002  96.400002       0
1528969080      96.470001  96.400002       0
1528969140      96.400002  96.400002       0
1528969200      96.400002  96.400002       0


In [123]:
times = sorted(main_df.index.values)
last_5pct = times[-int(0.05*len(times))]

validation_main_df = main_df[(main_df.index >= last_5pct)]
main_df = main_df[(main_df.index < last_5pct)]

In [124]:
#preprocess_df(main_df)
train_x, train_y = preprocess_df(main_df)
validation_x, validation_y = preprocess_df(main_df)

            BTC-USD_close  BTC-USD_volume  LTC-USD_close  LTC-USD_volume  \
time                                                                       
1528969140      -0.002513       -0.078919      -0.540429       -0.117090   
1528969200      -0.229992        0.003410       0.004048       -0.118307   
1528969260      -0.000830       -0.074039       0.004048       -0.072997   
1528969320       0.116299       -0.049626       0.004048       -0.108241   
1528969380       0.000937       -0.082052       0.004048       -0.048516   

            ETH-USD_close  ETH-USD_volume  BCH-USD_close  BCH-USD_volume  \
time                                                                       
1528969140       0.004601       -0.047838       0.011062       -0.006217   
1528969200       0.004601       -0.045377       0.257552       -0.005884   
1528969260       0.021273       -0.032900       0.265415       -0.006200   
1528969320       0.004601       -0.049530       0.455951       -0.005898   
1528969380 

/var/folders/1z/4vxcctj13w70p7lpfdqll_hw0000gn/T/ipykernel_49942/3046434640.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df =df.drop('future', 1)
/var/folders/1z/4vxcctj13w70p7lpfdqll_hw0000gn/T/ipykernel_49942/3046434640.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df =df.drop('future', 1)


            BTC-USD_close  BTC-USD_volume  LTC-USD_close  LTC-USD_volume  \
time                                                                       
1528969140      -0.002513       -0.078919      -0.540429       -0.117090   
1528969200      -0.229992        0.003410       0.004048       -0.118307   
1528969260      -0.000830       -0.074039       0.004048       -0.072997   
1528969320       0.116299       -0.049626       0.004048       -0.108241   
1528969380       0.000937       -0.082052       0.004048       -0.048516   

            ETH-USD_close  ETH-USD_volume  BCH-USD_close  BCH-USD_volume  \
time                                                                       
1528969140       0.004601       -0.047838       0.011062       -0.006217   
1528969200       0.004601       -0.045377       0.257552       -0.005884   
1528969260       0.021273       -0.032900       0.265415       -0.006200   
1528969320       0.004601       -0.049530       0.455951       -0.005898   
1528969380 

In [127]:
model = Sequential()
model.add(LSTM(128, input_shape = (train_x.shape[1:]), return_sequences = True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(128, input_shape = (train_x.shape[1:]), return_sequences = True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(LSTM(128, input_shape = (train_x.shape[1:])))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32, activation = "relu"))
model.add(Dropout(0.2))

model.add(Dense(2, activation = "softmax"))

model.compile(loss='sparse_categorical_crossentropy', optimizer = "adam", metrics = ["accuracy"])

model.fit(train_x, train_y, batch_size = batch_size, epochs = epochs, validation_data = (validation_x, validation_y))


2022-02-04 12:47:57.542584: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/10


2022-02-04 12:47:58.776055: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-04 12:47:59.283596: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-04 12:48:00.480821: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-04 12:48:00.677762: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-04 12:48:02.733147: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-04 12:48:04.089366: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-04 12:48:04.260958: I tensorflow/core/grappler/optimizers/cust

1082/1082 [==============================] - ETA: 0s - loss: 0.7151 - accuracy: 0.5261

2022-02-04 12:49:27.119610: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-04 12:49:27.273015: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-04 12:49:27.381399: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-04 12:49:27.486664: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1082/1082 [==============================] - 120s 105ms/step - loss: 0.7151 - accuracy: 0.5261 - val_loss: 0.6888 - val_accuracy: 0.5397
Epoch 2/10
1082/1082 [==============================] - 110s 102ms/step - loss: 0.6904 - accuracy: 0.5347 - val_loss: 0.6861 - val_accuracy: 0.5486
Epoch 3/10
1082/1082 [==============================] - 113s 104ms/step - loss: 0.6903 - accuracy: 0.5310 - val_loss: 0.6936 - val_accuracy: 0.5025
Epoch 4/10
1082/1082 [==============================] - 113s 105ms/step - loss: 0.6934 - accuracy: 0.5056 - val_loss: 0.6927 - val_accuracy: 0.5185
Epoch 5/10
1082/1082 [==============================] - 113s 105ms/step - loss: 0.6934 - accuracy: 0.5025 - val_loss: 0.6931 - val_accuracy: 0.5057
Epoch 6/10
1082/1082 [==============================] - 115s 107ms/step - loss: 0.6933 - accuracy: 0.5047 - val_loss: 0.6933 - val_accuracy: 0.5025
Epoch 7/10
1082/1082 [==============================] - 115s 107ms/step - loss: 0.6933 - accuracy: 0.5042 - val_loss: 0.693